In [5]:
from dotenv import load_dotenv
import enum
import instructor
from openai import OpenAI
import os
from pathlib import Path
from pprint import pprint as pp
from pydantic import BaseModel, Field, StringConstraints, conlist, field_validator
from typing import List, Union, Iterable
from typing_extensions import Annotated

In [3]:
# load API key

dotenv_path = Path(r"C:\Storage\python_projects\ashvin\.env")
load_dotenv(dotenv_path=dotenv_path)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# main constants

GPT_MODEL = "gpt-4o" # points to latest GPT model

#instantiate client
client = instructor.from_openai(OpenAI(), mode=instructor.Mode.TOOLS)

In [4]:
# query plan primitives

class QueryType(str, enum.Enum):
    """Enumeration representing the types of queries that can be asked to a question answer system."""

    SINGLE_QUESTION = "SINGLE"
    MERGE_MULTIPLE_RESPONSES = "MERGE_MULTIPLE_RESPONSES"

class Query(BaseModel):
    """Class representing a single question in a query plan."""

    id: int = Field(..., description="Unique id of the query")
    question: str = Field(
        ...,
        description="Question asked using a question answering system",
    )
    dependencies: List[int] = Field(
        default_factory=list,
        description="List of sub questions that need to be answered before asking this question",
    )
    node_type: QueryType = Field(
        default=QueryType.SINGLE_QUESTION,
        description="Type of question, either a single question or a multi-question merge",
    )

class QueryPlan(BaseModel):
    """Container class representing a tree of questions to ask a question answering system."""

    query_graph: List[Query] = Field(
        ..., description="The query graph representing the plan"
    )

    def _dependencies(self, ids: List[int]) -> List[Query]:
        """Returns the dependencies of a query given their ids."""

        return [q for q in self.query_graph if q.id in ids]

In [14]:
class FAQ(BaseModel):
    """Class for a single question and answer pair"""

    question: str
    answer: str

In [7]:
# revised wrapper

def wrapper(
    system_prompt: str | None = None, 
    user_prompt: Union[str, list] | None = None, 
    response_model: BaseModel | None = None, 
    max_retries: int = 3, 
    additional_messages: Union[str, List[str]] | None = None
):
    """Wrapper function to generate LLM completion"""
    messages = []

    # Add system prompt if provided
    if system_prompt is not None:
        messages.append({"role": "system", "content": system_prompt})

    # Add additional messages before user_prompt
    if additional_messages is not None:
        if isinstance(additional_messages, list):
            for message in additional_messages:
                messages.append({"role": "user", "content": message})
        else:
            messages.append({"role": "user", "content": additional_messages})

    # Add user context if provided
    if user_prompt is not None:
        if isinstance(user_prompt, list):
            for context in user_prompt:
                messages.append({"role": "user", "content": context})
        else:
            messages.append({"role": "user", "content": user_prompt})

    # Generate the completion
    completion = client.chat.completions.create(
        model=GPT_MODEL,
        response_model=response_model,
        max_retries=max_retries,
        messages=messages
    )
    return completion


In [24]:
faq_system_prompt = """
You are a world class question and answer algorithm capable of creating exceptionally clear, concise and precise questions and answering them.
Before you create the question-answer pairs, think step-by-step to get a better understanding of the problem, utilising any context available to you.
Your style guide is to follow the author Raymond carver for simplicity and clarity in composing questions and answers.
"""

In [25]:
faq_user_prompt = """
What are a detailed list of additional question and answer pairs that may be asked of a new to be formed AI platform team in a B2B SAAS company? 
I want precise and clearly understandable questions and answers. Do not replicate any question-answer pairs from the context.
"""

In [12]:
# response with additional messages as context 

additional_messages_1 = """
Platform

Innovations are adopted quickly if they are easy to make, easy to use, fit existing patterns, and have visible results. Generative AI has abruptly changed the innovation landscape by accelerating the utility and availability of AI products and services.
While the vast majority of consumers and companies do not make technology, they are actively searching for value in this space. Consequently, technology companies are trying to implement AI features that create value for their customers as fast as possible.
The long tail of B2B SaaS companies has a unique set of challenges. Many of these companies have enterprise client traction and skilled development teams but lack extra capital and in-house AI/ML resources, making it hard to bootstrap production grade AI capabilities and features.


Goals
A dedicated platform team helps B2B SaaS companies stay competitive by quickly matching competitors' advances and meeting client expectations for new experiences.
This team drives innovation, positions the company as a leader in AI, and reduces risk by providing systemic infrastructure and resources to prevent project failures.
It specifically helps:
Product teams launch AI features
Business teams automate workflows
Leaders unify AI strategy, standards and voice
Managers make data-driven decisions
Clients adopt third-party AI products
Staff upskill AI knowledge and fluency
Ultimately, platform teams shape organisational culture, ensuring that responsibility and enthusiasm for AI are shared by everyone, not just a small group of evangelists.
Good platform teams function as skunkworks, rapid execution engines, and guardrails, ensuring the company quickly and safely integrates AI across all operations in order to benefit clients and establish AI leadership amongst competitors.


Ideas
When bootstrapping platform teams, only speed and outcomes matter. Fast, cheap value creation determines whether platforms continue or are killed off because resources are limited, track records nonexistent and executive patience is thin.
Platform teams must act for all functions and departments in a B2B SaaS company. They should tackle cross disciplinary needs - at least product, design and engineering to start with - and champion fairness - equitable platform access to all products - as foundational principles for success.
Focusing on both technical tooling and service delivery is important, as the latter reduces early bottlenecks the most.
In this environment, prioritisation is king and selecting the correct set of valuable and feasible initiatives to iterate on and build company confidence is crucial.
Platform teams empower executives with a robust reporting framework that simplifies tracking, optimises resource allocation, and sharpens decision-making regarding AI initiatives.
Goal 1 : Help product teams launch AI features


KPI
Bottleneck
Actions
Write PRD in 1 day 
Lack of competitor awareness and AI expertise slows opportunity identification
1. Searchable, updated competitor and client database of AI features

2. Custom GPT for rapid ideation and document formatting
Develop prototype in 1 week
Lack of tooling slows conversion from idea to PoC
1. Curated, ready-to-go notebooks with composable components and pipelines to bootstrap a project

2. [Optional] Buy a low/no-code service to help non-engineers prototype
Create pitch in 1 day

Approval decision in 1 hour
Lack of standard templates, explicit criteria and quick feedback slows generation and evaluation
1. AI-powered templates to auto-construct pitches and provide instant feedback for human editing

2. Automated evaluation reports and pipelines to expedite committee approval

3. Access to platform team experts for direct consultation and recommendations


Build AI feature in 6 weeks
Setting up infrastructure, experimenting, and applying advanced techniques takes too much time
1. Maintain a quick-start list of managed services infrastructure for zero friction project setup and deployment

2. Provide an out-of-the-box observability platform for automatic monitoring and evaluations

3. Abstract technical infrastructure, design patterns and best practices around advanced features like RAG, Fine Tuning, Agent, Multimodal

4. Synthetic data pipelines for bootstrapping teams working with confidential data

5. Dedicated platform resources (e.g. engineers, designer) pairing with product teams to accelerate delivery

6. Provide UI/UX reviews, advice, starter Figma web components or AI design systems informed by best practices

7. Maintain a risk and safety register for the feature informed by AI best practices and frameworks
Compliance check in 1 day
Staying updated with regulatory frameworks, assessing their impact on product features, and preparing external documents is time-consuming.
1. Maintain a searchable, updated compliance directory to track and update local, state, national, and international regulatory frameworks.

2. Use RAG-based rubric evaluation for feature compliance

3. Automated drafting of approval request letters to appropriate stakeholders
GTM in 1 week
Lack of marketing resources and inconsistent AI messaging across teams delays launches
1. Automate release notes

2. Draft user documents pack, including how-to guides and support FAQs

3. Generate draft marketing messages, collateral and standard company templates/assets


Post-Launch Maintenance for 3 months
Maintaining AI features post-launch is resource-intensive, requiring continuous monitoring and a different approach for generative AI's probabilistic nature
1. Dedicated hypercare by AI platform team to handle bugs, fixes and updates for 3 months post launch

2. Set up automated monitoring systems to track AI performance and detect issues early.

3. Providing a BAU service request model for ongoing/complex issues



We need a wartime platform squad to stay ahead. Wartime squads move fast, cut red tape, and prioritise impact. They focus on speed and value, driving the company forward by launching AI features quickly. Leaders make tough calls to keep projects on track.
This team works seamlessly with product and business teams, enabling them and ruthlessly prioritising the best ideas. They eliminate bureaucracy, integrate new AI tools swiftly, and handle issues head-on. They ensure compliance and safety standards are met without slowing down progress.
Unlike peacetime squads that maintain stability, wartime squads thrive in urgency. They deliver immediate results and keep the company ahead of competitors. Quick decisions and a relentless focus on outcomes make them essential for AI leadership and rapid innovation.
"""

additional_messages_2 = """
FAQ
**Q:** Do AI platform teams lack the domain knowledge of specific products, making them unsuitable substitutes for product-specific engineering teams?


**A:** Yes, AI platform teams lack product-specific knowledge. They're support, not a substitute. To fix this, have the platform team act as a drop-in task force during critical phases, integrating AI while the product squad provides domain expertise. Alternatively, second selected product engineers from a squad to the platform team for a brief period - e.g. six weeks - to share domain knowledge and tailor AI solutions effectively.


**Q:** Will the product team be left to maintain the AI system alone after launch?


**A:** No. The platform team offers three months of hypercare for bugs and updates. Plus, the product team has access to observability tools to monitor performance and catch issues early. The platform team will also offer a BAU service request model for ongoing/complex issues.


**Q:** Will relying on the platform team create bottlenecks and turn them into gatekeepers?


**A:** No. The platform team is designed to be an enabler, not a gatekeeper. They prevent bottlenecks by embedding engineers with product teams during critical phases, offering self-service AI tools, and automated processes. The platform team also maintains clear communication and sets SLAs for quick response time. This ensures they support innovation and efficiency, rather than hindering progress.


**Q:** Will product engineers miss out on learning AI if platform engineers handle the builds, and will knowledge leave with them?


**A:** No. Platform engineers work closely with product teams, provide hands-on training, and keep detailed documentation. Product engineers involved in builds will teach their squad, ensuring everyone learns and retains AI skills.
**Q:** If I don't get approved to work with the platform team, does my AI journey end?


**A:** No. You can still use self-service tools, resources, and training from the platform team. Product teams keep decision-making autonomy and can build AI features on their own. Approved projects get embedded engineers and time-bound support, but this doesn't remove always-available features or limit reapplying for more support.


**Q:** How do you promote equitable access to platform resources across departments and products?
**A:** We set clear rules for resource allocation, ensure transparency, and provide self-service tools and training. Open communication helps manage competition, and all teams get equal access. Approved projects get extra support, but everyone can always use available features and reapply for more help.
**Q:** How do we measure the platform team's success?


**A:** We use practical metrics: time-to-market for AI features, the number of successful AI launches, user feedback, and AI feature funnel conversion from ideas to approved projects. This ensures our efforts provide real benefits to users and improve company results.


**Q:** How do we measure the success of the AI features?


**A:** The product team tracks user adoption, feedback, performance, and ROI. The platform team helps with tools for monitoring, synthetic data, and risk management. They also provide training, support, and regular reviews to ensure AI features deliver real value and meet business goals.


**Q:** What happens if product teams don’t cooperate with the platform team, or vice versa?


**A:** The platform team is an optional enabler, offering a range of services as needed. For approved projects with embedded engineers, cooperation is mandatory and ensured through formal agreements endorsed by managers and executives. These agreements include clear requirements, conditions, SLAs, and deliverables, ensuring alignment and accountability, which are critical for achieving company and user outcomes.


**Q:** How does the platform team handle both innovation and BAU (Business As Usual) tasks simultaneously?


**A:** The platform team manages both by dedicating 70% of their resources to innovation and 30% to BAU tasks. Given the newness of the platform team, this balance might shift as the environment and products mature, allowing for adjustments based on evolving needs and priorities.


**Q:** What if the platform team has too many critical projects to work on?


**A:** Managers and executives decide which projects to prioritize, based on the platform team's advice. We have a set ratio of resources to projects to manage capacity. This way, the most critical work gets done first.


**Q:** What if there's a critical issue post-launch that the product team or platform team can't handle?


**A:** We build AI features with feature flags, allowing us to disable them if needed. The platform team maintains a risk and safety register and ensures quick rollback provisions at the code level. If a critical issue arises that the product team or platform team can't manage, we activate existing mechanisms for P1 incidents, including customer communications by senior engineers, managers, and support teams.


**Q:** How do we handle the risk of burnout and ensure sustainability if the platform team is constantly working in high-pressure sprints?


**A:** The platform team operates in a high-pressure environment with a clear mission to accelerate AI feature launches. To manage burnout, we maintain a 70-30 split between innovation and BAU tasks, and rotate team members between high-intensity projects and less demanding tasks. We recruit team members with this demanding mission in mind from the outset, ensuring they are prepared for the challenges and committed to delivering high performance.


**Q:** What happens if there's a V2 feature idea after launch and hypercare?


**A:** If a V2 feature idea arises post-launch and hypercare, it goes back through the standard stage gates. For already launched products, we have an accelerated stage gate process. Due to resource limits, product teams can also proceed independently or use a day expertise consultancy model outside of the six-week drop-in period for additional support.
"""

additional_messages = [additional_messages_1, additional_messages_2]




In [26]:
response = wrapper(
    system_prompt=faq_system_prompt,
    user_prompt=faq_user_prompt,
    response_model=Iterable[FAQ],
    max_retries=3,
    additional_messages=additional_messages
)

In [27]:
for index, faq in enumerate(response, start=1):
    print(f"Question {index} : {faq.question}")
    print(f"Answer {index} : {faq.answer}")


Question 1 : What is the primary role of the AI platform team in a B2B SaaS company?
Answer 1 : The primary role of the AI platform team is to rapidly integrate AI capabilities across the company's products and services, ensure efficient use of AI resources, support product teams, and enable quick time-to-market for AI features.
Question 2 : How does the AI platform team prioritize projects?
Answer 2 : Project prioritization is based on impact, feasibility, and available resources. Managers and executives make decisions with input from the platform team to ensure the most critical and valuable projects are addressed first.
Question 3 : What kind of support does the platform team offer during the prototyping phase?
Answer 3 : The platform team provides ready-to-go notebooks, composable components, managed services for quick setup, and optional low/no-code services to help non-engineers prototype rapidly.
Question 4 : How does the platform team ensure compliance with AI regulations?
Answ